# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [ ]:
from samples.coco import coco

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.DETECTION_NMS_THRESHOLD = 0.5
config.RPN_NMS_THRESHOLD = 0.5
# config.MAX_GT_INSTANCES = 50
# config.BACKBONE = "resnet50"
# config.IMAGE_MIN_DIM = 256
# config.IMAGE_MAX_DIM = 256
# config.GPU_COUNT = 1
# config.IMAGES_PER_GPU = 1
# config.TRAIN_ROIS_PER_IMAGE = 50
config.display()

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [ ]:
# Load a random image from the images folder
IMAGE_DIR = '/home/eliab/exp/reindex_bags/data/rgb/'
SEMANTIC_DIR = '/home/eliab/exp/reindex_bags/data/semantic/'

if not os.path.exists(SEMANTIC_DIR):
    os.makedirs(SEMANTIC_DIR)

file_names = next(os.walk(IMAGE_DIR))[2]
file_names.sort(key=lambda x:int(x[:-4]))


import cv2
for file_name in file_names:
    images = []
    images.append(cv2.imread(IMAGE_DIR + file_name))
    print('Loading %s' %(file_name),end='\r')
    
    # Run detection
    results = model.detect(images, verbose=0)

    # Visualize results
    r = results[0]['masks']
    mask = np.zeros((r.shape[0],r.shape[1]))

    for i in range(r.shape[2]):
        index = np.where(r[:,:,i]==True)
        pixel_list = list(zip(index[0],index[1]))
        for x,y in pixel_list:
            mask[x,y] = i+1
    
    fn = str(int(file_name[:-4])-1).zfill(6) + '.txt' # zfill can be removed
    # Save the semantic results
    np.savetxt(SEMANTIC_DIR + fn, mask, fmt='%d', delimiter=' ')
    
    img_fn = os.path.join(SEMANTIC_DIR, fn[:-4] +'.png')
    # save the images
    visualize.display_instances(img_fn, images[0], results[0]['rois'], results[0]['masks'], results[0]['class_ids'], class_names, results[0]['scores'])


*** No instances to display *** 



In [ ]:
visualize.display_instances(images[0], results[0]['rois'], results[0]['masks'], results[0]['class_ids'], class_names, results[0]['scores'])

# display_instances(image, boxes, masks, class_ids, class_names, scores=None, title='', figsize=(16, 16), ax=None, show_mask=True, show_bbox=True, colors=None, captions=None)
#     boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
#     masks: [height, width, num_instances]
#     class_ids: [num_instances]
#     class_names: list of class names of the dataset
#     scores: (optional) confidence scores for each box
#     title: (optional) Figure title
#     show_mask, show_bbox: To show masks and bounding boxes or not
#     figsize: (optional) the size of the image
#     colors: (optional) An array or colors to use with each object
#     captions: (optional) A list of strings to use as captions for each object

In [ ]:
# Visualize results
for k in range(5):
    r = results[k]['masks']
    mask = np.zeros((r.shape[0],r.shape[1]))

    for i in range(r.shape[2]):
        index = np.where(r[:,:,i]==True)
        pixel_list = list(zip(index[0],index[1]))
        for x,y in pixel_list:
            mask[x,y] = i+1

    masked_img = visualize.display_instances(images[k], results[k]['rois'], r, results[k]['class_ids'], class_names, results[k]['scores'])

    np.savetxt('semantic/'+str(k)+'.txt',mask, fmt='%d', delimiter=' ')
    cv2.imwrite('semantic/'+str(k)+'.png',masked_img)

In [ ]:
result = np.load('/home/cxu/new_data/exp1/Viewport0_occluded/instance/1.npy',allow_pickle=True)